In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
print("hi")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving spotify_millsongdata.csv to spotify_millsongdata.csv


In [ ]:
df = pd.read_csv("spotify_millsongdata.csv")


In [ ]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [ ]:
df.shape

(57650, 4)

In [ ]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [ ]:
df =df.sample(20000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
df.head(10)

,artist,song,text
0,Nazareth,Pop The Silo,The phone call was wrong \r\nThe i.b.m.'s hav...
1,Richard Marx,Nothing Left Behind Us,You never know what we could be in store for ...
2,Kylie Minogue,Disco Down,I've been thinking reminiscing \r\n'bout the ...
3,Phil Collins,We Said Hello Goodbye,We said goodbye to a dear old friend \r\nAnd ...
4,The Weeknd,Acquainted,[Verse 1] \r\nBaby you smell good \r\nCause ...
5,Hillsong,More Than,Your eternal word formed the universe \r\nAnd...
6,Alabama,As Right Now,You're still the lady of my dreams \r\nYou're...
7,Kate Bush,Sexual Healing,"Uuh, baby, I'm hot just like an oven \r\nI ne..."
8,Status Quo,No Contract,"I don't want your lovin', baby, I won't give y..."
9,Faith Hill,A Baby Changes Everything,"Teenage girl, much too young \r\nUnprepared f..."


In [ ]:
df['text'][0]

"That there  \r\nThat's not me  \r\nI go  \r\nWhere I please  \r\n  \r\nI walk through walls  \r\nI float down the Liffey  \r\nI'm not here  \r\nThis isn't happening  \r\nI'm not here  \r\nI'm not here  \r\n  \r\nIn a little while  \r\nI'll be gone  \r\nThe moment's already passed  \r\nYeah it's gone  \r\nAnd I'm not here  \r\nThis isn't happening  \r\nI'm not here  \r\nI'm not here  \r\n  \r\nStrobe lights and blown speakers  \r\nFireworks and hurricanes  \r\nI'm not here  \r\nThis isn't happening  \r\nI'm not here  \r\nI'm not here\r\n\r\n"

In [ ]:
df.shape

(20000, 3)

In [ ]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [ ]:
similarity[0]

array([1.        , 0.02257669, 0.02238161, ..., 0.02422256, 0.02410638,
       0.00297171])

In [ ]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [ ]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [ ]:
# Assuming you have a function named 'recommendation' and have defined 'df' and 'similarity'
recommended_songs = recommendation(df[df['song'] == 'Crying Over You'])
print(recommended_songs)


<ipython-input-54-6aa5d0a0422a>:2: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  idx = df[df['song'] == song_df].index[0]


["Find Out What's Happening", 'Again', "I Won't Let It Happen", "Even When You're Far Away", 'Again', "Nothing's Happening By The Sea", 'If It Happens Again', 'Gone (The Vacuous House Mix)', 'Til It Happens To You', 'Anything Could Happen', 'Here Is The House', 'Everything Happens For The Best', 'Make It Happen', 'Action', "Don't You Feel Small", 'Full Moon Night', "Don't You Even Care", 'Could It Happen To Me?', 'I Still Care What Happens To You', 'Good Things']


In [1]:
import pickle